In [6]:
import pandas as pd
import numpy as np
import re,glob,os,secrets
import urllib.parse
import urllib.request
from multiprocessing.pool import ThreadPool
import urllib
import pandas as pd
import operator
import functools
from operator import methodcaller

URL = 'https://www.uniprot.org/uploadlists/'

def create_chunks(list_name, n):
    for i in range(0, len(list_name), n):
        yield list_name[i:i + n]
        
def mapping_ids(ids2map, source_fmt='P_GI',target_fmt='ACC', output_fmt='tab'):
    if hasattr(ids2map, 'pop'):
            ids2map = ' '.join(ids2map)
    params = {
    'from': source_fmt,
    'to': target_fmt,
    'format': output_fmt,
    'query': ids2map
    }

    data = urllib.parse.urlencode(params)
    data = data.encode('utf-8')
    req = urllib.request.Request(URL, data)
    with urllib.request.urlopen(req) as f:
        response = f.read().decode('utf-8')
    return response

def concat_dfs(path,extension):
    extension = extension
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    
    combined_dfs = pd.concat([pd.read_table(f) for f in all_filenames ])
    
    return combined_dfs

    

In [7]:
df_besthits = pd.read_table("C:/Users/eulle/Documents/besthits/besthits_ROCHA2.csv")

columns_csv = ['qseqid' ,'sseqid' ,'pident' ,'length','mismatch','gapopen','qstart' ,'qend' ,'sstart' ,'send' ,'evalue' ,'bitscore']
row_values = pd.Series(df_besthits.columns)
df_besthits = pd.DataFrame(np.vstack([df_besthits.columns,df_besthits]))
df_besthits.columns = columns_csv


In [8]:
gi_number = [re.findall(r"(?<=GI:|gi:).*?(?=\||p)", ii[1]['sseqid']) for ii in df_besthits.iterrows()]
gi_number = functools.reduce(operator.iconcat, gi_number, [])
gi_number = list(set(gi_number))

uniprot_id = [re.findall(r"(?<=BL:|ot:).*?(?=\||p)",ii[1]['sseqid']) for ii in df_besthits.iterrows()]
uniprot_id = functools.reduce(operator.iconcat, uniprot_id, [])
uniprot_id = list(set(uniprot_id))

chunk_gi = list(create_chunks(gi_number,1000))
chunk_uniprot = list(create_chunks(uniprot_id,1000))

In [9]:
# Criar uma função para escrever os arquivos
map_gi = ThreadPool(10).imap(mapping_ids,chunk_gi)
map_uniprot = ThreadPool(10).imap(functools.partial(mapping_ids,source_fmt='ACC',target_fmt='KO_ID'),chunk_uniprot)

for index,ii in enumerate(map_gi): 
    with open('C:/Users/eulle/Documents/gi_number-uni/'+ str(index)+'.csv', 'w') as file:
        file.write(ii)
        
for index,ii in enumerate(map_uniprot): 
    with open('C:/Users/eulle/Documents/uniprot-ko_id/'+ str(index)+'.csv', 'w') as file:
        file.write(ii)

In [10]:
# Pensar em uma forma de melhorar esses downloads
gi_uniprot_concat = concat_dfs(os.chdir("C:/Users/eulle/Documents/gi_number-uni/"),"csv")
gi_uniprot_concat = gi_uniprot_concat.drop_duplicates("From")
gi_uniprot_concat = gi_uniprot_concat.reset_index(drop=True)
gi_uniprot_concat.columns = ["GI","Uniprot_ID"]

gi_uniprot_concat.to_csv("df_gi_uniprot.csv",sep="\t",encoding="utf-8",index=False)

list_gi_uniprot = list(gi_uniprot_concat["Uniprot_ID"])
chunk_uniprot = list(create_chunks(list_gi_uniprot,1000))
map_uniprot = ThreadPool(10).imap(functools.partial(mapping_ids,source_fmt='ACC',target_fmt='KO_ID'),chunk_uniprot)

for index,ii in enumerate(map_uniprot): 
    with open('C:/Users/eulle/Documents/uniprot-ko_id/'+ secrets.token_hex(15) +'.csv', 'w') as file:
        file.write(ii)



In [10]:
uniprot_ko_concat = concat_dfs(os.chdir("C:/Users/eulle/Documents/uniprot-ko_id/"),"csv")
uniprot_ko_concat.columns = ["Uniprot_ID","KO"]
uniprot_ko_concat = uniprot_ko_concat.reset_index(drop=True)

uniprot_ko_concat.to_csv("df_ko_ids_CoSQG.csv",sep="\t",encoding="utf-8",index=False)